In [ ]:
# Import relevant Built-in Libraries & Functions
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope

In [ ]:
# Import Models
from models.unets import Unet2D
from models.FCN import FCN_Vgg16_16s
from models.SegNet import SegNet

In [ ]:
# Import custom Utilities
from utils.learning.metrics import dice_coef, precision, recall
from utils.learning.losses import dice_coef_loss
from utils.io.data import DataGen, save_results, save_history, load_data

In [ ]:
# Define variables
input_dim_x=224
input_dim_y=224
n_filters = 32

In [ ]:
# Data Generation variables
dataset = 'Medetec_foot_ulcer_224'
data_gen = DataGen('./data/' + dataset + '/', split_ratio=0.2, x=input_dim_x, y=input_dim_y)

In [ ]:
# UNet Model
unet2d = Unet2D(n_filters=n_filters, input_dim_x=None, input_dim_y=None, num_channels=3)

In [ ]:
# FCN Model
model, model_name = FCN_Vgg16_16s(input_shape=(input_dim_x, input_dim_y, 3))

In [ ]:
# SegNet Model
segnet = SegNet(n_filters, input_dim_x, input_dim_y, num_channels=3)
model, model_name = segnet.get_SegNet()

In [ ]:
# Variables for training
batch_size = 2
epochs = 2000
learning_rate = 1e-4
loss = 'binary_crossentropy'

In [ ]:
# Define the state for Early-Stopping
es = EarlyStopping(monitor='val_dice_coef', patience=200, mode='max', restore_best_weights=True)

In [ ]:
# Model Summary
model.summary()

In [ ]:
# Model Compilation with Optimizer
model.compile(optimizer=Adam(lr=learning_rate), loss=loss, metrics=[dice_coef, precision, recall])

In [ ]:
# Training Process
training_history = model.fit_generator(data_gen.generate_data(batch_size=batch_size, train=True),
                                       steps_per_epoch=int(data_gen.get_num_data_points(train=True) / batch_size),
                                       callbacks=[es],
                                       validation_data=data_gen.generate_data(batch_size=batch_size, val=True),
                                       validation_steps=int(data_gen.get_num_data_points(val=True) / batch_size),
                                       epochs=epochs)

In [ ]:
# Save the Trained Filters & Training History
save_history(model, model_name, training_history, dataset, n_filters, epochs, learning_rate, loss, color_space='RGB',
             path='./training_history/')